In [46]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
df = pd.read_csv('flight_df.csv')
df=df.drop(columns='Flight info')
df=df.drop(columns='Flight time')
df['Date']=df['Date']*100
df['Price']=df['Price'].str.split(',').str[0].astype(int)*1000+df['Price'].str.split(',').str[1].astype(int)
df.head()

,Date,Day,Airline,Departure time,Departure airport,Arrival time,Arrival airport,Price
0,601.0,목,ANA,19:55,GMP,22:15,HND,329700
1,601.0,목,일본항공,07:50,GMP,09:55,HND,331100
2,601.0,목,일본항공,19:20,GMP,21:30,HND,331100
3,601.0,목,아시아나항공,15:30,GMP,17:35,HND,334300
4,601.0,목,아시아나항공,19:50,GMP,22:00,HND,334300


In [24]:
#데이터 인코딩
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

df_encoded= df
df_encoded['Date']=(df_encoded['Date'])//100+((df_encoded['Date']).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
df_encoded['Arrival time']=df_encoded['Arrival time'].str.split(':').str[0].astype(int)
df_encoded['Departure time']=df_encoded['Departure time'].str.split(':').str[0].astype(int)

df_ohe=enc.fit_transform(df_encoded.drop(columns='Price')).toarray()


In [25]:
#테스트 트레인 데이터 분할
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

target = 'Price'
X_train, X_test, y_train, y_test  = train_test_split(
    df_ohe, df_encoded['Price'], test_size=0.2, random_state=1
)

In [26]:
#LR
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(mean_squared_error(y_pred, y_test, squared = False))
print(r2_score(y_test, y_pred))


109138.29011653773
0.45135656988901984


In [27]:
import pickle

#파라미터 저장
pkl_filename = "flight.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr, file)

In [53]:
df = pd.read_csv('flight_df.csv')
df=df.drop(columns='Flight info')
df=df.drop(columns='Flight time')
df['Date']=df['Date']*100
df['Price']=df['Price'].str.split(',').str[0].astype(int)*1000+df['Price'].str.split(',').str[1].astype(int)

#파라미터 불러오기
with open("flight.pkl", 'rb') as file:
    flightpredicter = pickle.load(file)

In [52]:
#if Random
random_data=df.sample(1)
actual_price=random_data['Price'].iloc[0]
random_data=random_data.drop(columns='Price')
input_data=[list(random_data.iloc[0])]

#input_data=[['601', '목', 'ANA', '19:55', 'GMP', '22:15', 'HND']]

print(input_data)

#입력 데이터 전처리
X=pd.DataFrame(input_data, columns=['Date','Day','Airline','Departure time','Departure airport','Arrival time','Arrival airport'])
X['Date']=X['Date'].astype(int)
X['Date']=(X['Date'])//100+((X['Date']).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
X['Arrival time']=X['Arrival time'].str.split(':').str[0].astype(int)
X['Departure time']=X['Departure time'].str.split(':').str[0].astype(int)
X_encoded=enc.transform(X).toarray()

#예측
result = flightpredicter.predict(X_encoded)
print('Actual Price:', actual_price, 'Predicted Price:', result)

[[617.0, '토', '에티오피아항공', '17:25', 'ICN', '20:15', 'NRT']]
Actual Price: 267332 Predicted Price: [278784.]


In [30]:
#OLS

from statsmodels.formula.api import ols
from statsmodels.regression.linear_model import OLS
model = OLS(y_train, X_train)
result = model.fit()
y_pred = result.predict(X_test)
print(r2_score(y_pred, y_test))

-0.10769468220073719


In [69]:
#Experiment

df = pd.read_csv('flight_df.csv')
df=df.drop(columns='Flight info')
df=df.drop(columns='Flight time')
#df=df.drop(columns='Departure time')
df=df.drop(columns='Arrival time')


####
enc = OneHotEncoder()

df_encoded= df
df_encoded['Date']=(df_encoded['Date']*100)//100+((df_encoded['Date']*100).mod(100)//7)/10 #월/일이 아니라 월/주로 분리
#df_encoded[' Arrival time']=df_encoded[' Arrival time'].str.split(':').str[0].astype(int)
df_encoded['Departure time']=df_encoded['Departure time'].str.split(':').str[0].astype(int)
df_encoded['Price']=df_encoded['Price'].str.split(',').str[0].astype(int)*1000

df_ohe=enc.fit_transform(df_encoded.drop(columns=' Price')).toarray()

#####
target = 'Price'
X_train, X_test, y_train, y_test  = train_test_split(
    df_ohe, df_encoded['Price'], test_size=0.2, random_state=1
)


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

print(r2_score(y_test, y_pred))


0.43144437486062814


In [ ]:
#Arrival 빼고: 0.43144437486062814
#Departure 빼고: 0.410816052709579